In [4]:
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv("layoffs_data.csv")

# Group by industry and date, then calculate average layoffs
grouped = df.groupby(["Industry", "Date"])["Laid_Off_Count"].mean().reset_index()

# Create line plot
fig = px.line(grouped, x="Date", y="Laid_Off_Count", color="Industry", title="Average Layoffs by Industry Over Time")
fig.show()
plot_html = fig.to_html(full_html=False)

In [11]:
df.Laid_Off_Count.info()

<class 'pandas.core.series.Series'>
RangeIndex: 2738 entries, 0 to 2737
Series name: Laid_Off_Count
Non-Null Count  Dtype  
--------------  -----  
1871 non-null   float64
dtypes: float64(1)
memory usage: 21.5 KB


In [12]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, Select
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.io import curdoc
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.themes import Theme

# Load and prepare data
df = pd.read_csv("layoffs_data.csv")
df['Year'] = pd.to_datetime(df['Date']).dt.year
grouped = df.groupby(['Year', 'Industry'])['Laid_Off_Count'].sum().reset_index()

# Define function to create document
def modify_doc(doc):
    source = ColumnDataSource(grouped[grouped['Year'] == grouped['Year'].min()])

    # Create plot
    p = figure(x_range=grouped['Industry'].unique(), plot_height=350, title="Total Layoffs by Industry")
    p.vbar(x='Industry', top='Laid_Off_Count', width=0.9, source=source)

    # Create dropdown
    select = Select(title="Year", value=str(grouped['Year'].min()), options=grouped['Year'].unique().astype(str).tolist())

    # Update function
    def update(attr, old, new):
        new_src = grouped[grouped['Year'] == int(select.value)]
        source.data = dict(
            Industry=new_src['Industry'],
            Laid_Off_Count=new_src['Laid_Off_Count']
        )
        p.title.text = f"Total Layoffs by Industry in {select.value}"

    select.on_change('value', update)

    doc.add_root(column(select, p))

# Create application
handler = FunctionHandler(modify_doc)
app = Application(handler)

# Save to file
output_file("barplot.html")
show(app)

AssertionError: 

In [14]:
grouped_company = df.groupby(['Industry', 'Company'])['Laid_Off_Count'].sum().reset_index()
top_industries = grouped_company.groupby('Industry')['Laid_Off_Count'].sum().nlargest(10).index
filtered_industries = grouped_company[grouped_company['Industry'].isin(top_industries)]
top_companies = filtered_industries.groupby('Industry').apply(lambda x: x.nlargest(10, 'Laid_Off_Count'))
top_companies.reset_index(drop=True, inplace=True)

In [15]:
top_companies

,Industry,Company,Laid_Off_Count
0,Consumer,Meta,21000.0
1,Consumer,Google,12000.0
2,Consumer,Twitter,3940.0
3,Consumer,Bytedance,3750.0
4,Consumer,Yahoo,1600.0
...,...,...,...
95,Travel,Sabre,1100.0
96,Travel,TripAdvisor,900.0
97,Travel,Sonder,750.0
98,Travel,MakeMyTrip,700.0


In [13]:
html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Job Layoffs</title>
    <style>
        body {{
            background-color: #f6f6f6;
            font-family: Arial, sans-serif;
        }}

        header {{
            text-align: center;
            font-size: 2em;
            margin: 5px;
        }}

        main {{
            width: 60%;
            margin: 0 auto;
            background-color: #ffffffc1;
            padding: 50px;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
            min-height: calc(100vh - 300px);
        }}

        footer {{
            text-align: center;
            padding: 20px 0;
            color: #777;
        }}
    </style>
</head>
<body>
    <header>
        <h1>Job Layoffs</h1>
    </header>
    <main>
        Test
        <div id="plot">
            <iframe src="http://localhost:8050/" width="100%" height="600"></iframe>
        </div>
    </main>
    <footer>
        Test
    </footer>
</body>
</html>
"""

# Write the HTML string to a file
with open('index.html', 'w') as f:
    f.write(html)